In [1]:
!pip install selenium

In [8]:
from selenium import webdriver
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

#Se necesita tener chromedriver instalado para que selenium funcione

In [3]:
def obtener_info(driver):
    #A los 5 seg de entrar a la pagina emerge un anuncio, entonces se espera 5seg y se cierra
    time.sleep(5)
    driver.find_element_by_xpath('//div[@class="slidedown-footer"]/button[@class="align-right secondary slidedown-button"]').click()
    
    
    vuelos = driver.find_elements_by_xpath('//li[@class="flight"]')
    print(f'Se encontraron {len(vuelos)} vuelos')
    print("Iniciando scraping...")
    info = []
    for vuelo in vuelos:
        #tiempos generales
        tiempos = obtener_tiempos(vuelo)
        #click boton de escalas
        boton_escalas = vuelo.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]/button')
        boton_escalas.click()
        #informacion tiempo escalas
        escalas = obtener_datos_escalas(vuelo)
        #cerramos ventana
        boton_cerrar = vuelo.find_elements_by_xpath('//div[@class="modal-header sc-dnqmqq cGfTsx"]/button')[0]
        boton_cerrar.click()
        #clickeamos vuelo para ver tarifas
        vuelo.click()
        precios = obtener_precios(vuelo)
        #cierro info
        vuelo.click()
        
        info.append({'precios':precios, 'tiempos':tiempos, 'escalas':escalas})
    
    return info

In [4]:
def obtener_tiempos(vuelo):
    '''
    Dado un vuelo retorna un diccionario con los horarios de salida y llegada incluyendo duracion.
    '''
    hora_salida = vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')
    hora_llegada = vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')
    tiempo_vuelo = vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')

    data_dict={
    'hora_salida':hora_salida,
    'hora_llegada':hora_llegada,
    'tiempo_vuelo':tiempo_vuelo
    }
    
    return data_dict

In [5]:
def obtener_datos_escalas(vuelo):
    '''
    Dado un vuelo retorna una lista de diccionarios con la informacion
    de las escalas.
    '''
    segmentos = vuelo.find_elements_by_xpath('//div[@class="sc-hZSUBg gfeULV"]/div[@class="sc-cLQEGU hyoued" or @class="sc-cLQEGU dnKRNG"]')
    info_escalas = []
    for i, segmento in enumerate(segmentos):
        try:
            origen = segmento.find_element_by_xpath('.//div[@class="sc-gwVKww iahkEA"]/div/abbr').get_attribute("title")
            hora_salida = segmento.find_element_by_xpath('.//div[@class="sc-gwVKww iahkEA"]/div/time').get_attribute("datetime")
            destino = segmento.find_elements_by_xpath('.//div[@class="sc-bwCtUz iybVbT"][1]')[1].find_elements_by_xpath('.//abbr')[0].get_attribute("title")
            hora_llegada = segmento.find_elements_by_xpath('.//div[@class="sc-bwCtUz iybVbT"][1]')[1].find_elements_by_xpath('.//time')[0].get_attribute("datetime")
            duracion = segmento.find_element_by_xpath('.//span[@class="sc-esjQYD dMquDU"]/time').get_attribute("datetime")
            nro = segmento.find_element_by_xpath('.//div[@class="airline-flight-details"]/b').text
            modelo = segmento.find_element_by_xpath('.//div[@class="airline-flight-details"]/span[@class="sc-gzOgki uTyOl"]').text
            
            data_dict={
                'origen':origen,
                'hora_salida':hora_salida,
                'destino':destino,
                'hora_llegada':hora_llegada,
                'duracion':duracion,
                'nro':nro,
                'modelo':modelo }
            
            info_escalas.append(data_dict.copy())
            
        except Exception:
            pass
            
    
    return info_escalas

In [6]:
def obtener_precios(vuelo):
    precios = vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]')
    '''
    Dado un vuelo retorna una lista de diccionarios con las tarifas
    '''
    tarifas = []
    for precio in precios:
        nombre = precio.find_element_by_xpath('.//label').get_attribute("for")
        moneda = precio.find_element_by_xpath('.//span[@class="currency-symbol"]').text
        valor = precio.find_element_by_xpath('.//span[@class="value"]').text
        dict_tarifa = {nombre:{'moneda':moneda,'valor':valor}}
        tarifas.append(dict_tarifa)
    return tarifas

In [9]:
driver = webdriver.Chrome(ChromeDriverManager().install())
url = 'https://www.latam.com/es_ar/apps/personas/booking?fecha1_dia=01&fecha1_anomes=2021-05&fecha2_dia=22&fecha2_anomes=2021-05&from_city2=MAD&to_city2=BUE&auAvailability=1&ida_vuelta=ida_vuelta&vuelos_origen=Buenos%20Aires&from_city1=BUE&vuelos_destino=Madrid&to_city1=MAD&flex=1&vuelos_fecha_salida_ddmmaaaa=30/04/2021&vuelos_fecha_regreso_ddmmaaaa=22/05/2021&cabina=Y&nadults=1&nchildren=0&ninfants=0&cod_promo=&stopover_outbound_days=0&stopover_inbound_days=0&application=#/url ='
driver.get(url)

delaymax = 10
try:
    #demora dinamica de max 10 segundo, pero si se detecta un vuelo deja de esperar.
    vuelo = WebDriverWait(driver, delaymax).until(EC.presence_of_element_located((By.XPATH, '//li[@class="flight"]')))
    print('La pagina termino de cargar')
    info_vuelos = obtener_info(driver)
except TimeoutException:
    print('La pagina tardo mucho en cargar')

driver.close()
info_vuelos



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Get LATEST driver version for 90.0.4430
Trying to download new driver from https://chromedriver.storage.googleapis.com/90.0.4430.24/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Facun\.wdm\drivers\chromedriver\win32\90.0.4430.24]


La pagina tardo mucho en cargar


NameError: name 'info_vuelos' is not defined